In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.chrome.options import Options
import json
import re

In [23]:
cases_links = []
all_span_tuples = []

total_pages = 2  # alterar para a quantidade correta

for page in range(1, total_pages + 1):
    url = f'https://radiopaedia.org/search?lang=us&page={page}&scope=cases&sort=date_of_publication'
    response = requests.get(url)
    content = response.content

    soup = BeautifulSoup(content, 'html.parser')

    elements = soup.find_all('a', class_='search-result-case')

    divs = soup.select('#search-results div.col-xs-12.search-results-listing a div.search-result-body div.search-result-description div.search-result-modalities')

    for div, element in zip(divs, elements):
        spans = div.find_all('span')
        span_texts = [span.text.strip() for span in spans]
        all_span_tuples.append(tuple(span_texts))

        if 'CT' not in span_texts and 'MRI' not in span_texts:
            link = element.get('href')
            cases_links.append('https://radiopaedia.org' + link)
            

In [24]:
options = Options()
options.headless = True  # Ativado para que o navegador não seja aberto.

driver = webdriver.Chrome(options=options)


In [25]:
for case in cases_links:
    driver.get(case)

    time.sleep(5)
    div_element = driver.find_element(By.CLASS_NAME, 'fa-clickable')
    div_element.click()

    time.sleep(5)
    driver.refresh()

    # Find the script element directly
    script_element = driver.find_element(By.XPATH, '//script[contains(text(), "var stackedImages")]')

    # Get the text content of the script element
    codigo_js = script_element.get_attribute('text')


    match = re.search(r'var stackedImages = (.*?);', codigo_js)

    if match:
        json_data = match.group(1)

        data = json.loads(json_data)
        image_info = []

        for entry in data:
            for image in entry['images']:
                info = {
                    'public_filename': image['public_filename'],
                    'plane_projection': image['plane_projection']
                }
                image_info.append(info)


        for info in image_info:
            print(f"Public Filename: {info['public_filename']}")
            print(f"Plane Projection: {info['plane_projection']}")
            print('-' * 30)
    else:
        print("Pattern not found.")

ElementClickInterceptedException: Message: element click intercepted: Element <div style="width: 24px; height: 24px; " class="fa-clickable">...</div> is not clickable at point (753, 704). Other element would receive the click: <iframe id="google_ads_iframe_/22152718,2190163/radiopaedia_adhesive_cases_0" name="google_ads_iframe_/22152718,2190163/radiopaedia_adhesive_cases_0" title="3rd party ad content" width="728" height="90" scrolling="no" marginwidth="0" marginheight="0" frameborder="0" role="region" aria-label="Advertisement" tabindex="0" style="border: 0px; vertical-align: bottom;" data-load-complete="true" data-google-container-id="5"></iframe>
  (Session info: chrome=118.0.5993.88)
Stacktrace:
#0 0x5632da920fb3 <unknown>
#1 0x5632da5f44a7 <unknown>
#2 0x5632da641ecd <unknown>
#3 0x5632da64026d <unknown>
#4 0x5632da63e028 <unknown>
#5 0x5632da63d097 <unknown>
#6 0x5632da631d59 <unknown>
#7 0x5632da65d942 <unknown>
#8 0x5632da631776 <unknown>
#9 0x5632da65db0e <unknown>
#10 0x5632d